# Patient scores

In [ ]:
#bond: questions 3,5,7,9
#goal: 1,4,8,11
#task: 2,6,10,12

In [90]:
import pandas as pd

df = pd.read_csv('/Users/rivka/Desktop/Final/Trentofeat_incl_patWAI2.csv', delimiter=',')

# List of column names 1-12
columns_to_check = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
columns_to_check_t = ['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10']

# Convert the specified columns to numeric, replacing invalid values with NaN
df[columns_to_check] = df[columns_to_check].apply(pd.to_numeric, errors='coerce')

# Check for non-float values in the specified columns
non_float_mask = ~df[columns_to_check].applymap(lambda x: isinstance(x, float))

# Find rows where any of the specified columns have non-float values
rows_with_non_floats = non_float_mask.any(axis=1)

# Remove rows with non-float values
df = df.drop_duplicates()
df = df.dropna(subset=columns_to_check, how='any')

# df = df.drop_duplicates(subset=['ppnr', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10'])
df = df.drop_duplicates(subset=['ppnr','1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])

# If needed, reset the index of the filtered DataFrame
df.reset_index(drop=True, inplace=True)
df.fillna(0, inplace=True)

In [73]:
import pandas as pd

# Assuming you have a DataFrame called 'df' with the desired columns
bond_columns = ['3', '5', '7','9']
goal_columns = ['1', '4', '8','11']
task_columns = ['2', '6', '10','12']

# Calculate the row-wise average for the specified columns
bond_df = df[bond_columns].mean(axis=1)
goal_df = df[goal_columns].mean(axis=1)
task_df = df[task_columns].mean(axis=1)



In [ ]:
df #45 rows

## RF

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn.impute import SimpleImputer

# Load data from CSV
all_df = df

# Convert numeric columns to float
numeric_columns = ['ppnr', 'session']
all_df[numeric_columns] = all_df[numeric_columns].astype(float)

# Group the rows by 'ppnr' and 'sessions' columns and calculate the mean for other columns
# df = all_df.groupby(['ppnr', 'session'], as_index=False).mean()
df = all_df

# Select features and target variable
features = df[[ 'mean_waittime', 'median_waittime', 'std_waittime','nr_of_turns', 'turns_therapist',
       'turns_speaker_patient', 'duration_therapist', 'duration_patient',
       'participation_equality', 'turn_level_freedom',
       'avgturnlength_therapist', 'avgturnlength_patient',
       'wordcount_speaker_therapist', 'wordcount_speaker_patient',
       'speechrate_speaker_therapist', 'speechrate_speaker_patient',
       'speechduration_therapist', 'speechduration_patient',
       'durationpercentage_therapist', 'durationpercentage_patient']]


for i in range(12):
    nr = str(i+1) #'t' +

    outcome = all_df[nr]
    print("WAI question:", nr)

    # Handle missing values in features using an imputer
    imputer = SimpleImputer(strategy='mean')
    features_imputed = imputer.fit_transform(features)

    # Train the model
    t = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
    t.fit(features_imputed, outcome)

    # Calculate out-of-bag MSE for Outcome 1
    oob_error = 1 - t.score(features_imputed, outcome)
    print("Out-of-Bag MSE for Outcome 1:", oob_error)

    # Get feature importances
    impOOB = t.feature_importances_

    # Plot feature importances
    plt.bar(range(len(impOOB)), impOOB)
    plt.title('Unbiased Predictor Importance Estimates')
    plt.xlabel('Predictor variable')
    plt.ylabel('Importance')
    plt.xticks(range(len(impOOB)), features.columns, rotation=90)
    # plt.show()

    # Check by predicting held back data
    ncells = features.shape[0]
    perc = int(0.8 * ncells)

    nboots = 10
    Acc = np.empty(nboots)
    for b in range(nboots):
        shuf = np.random.permutation(ncells)
        incl = shuf[:perc]
        holdback = shuf[perc:]

        Mdl = t.fit(features.iloc[incl], outcome.iloc[incl])
        label = Mdl.predict(features.iloc[holdback])
        Acc[b] = mean_squared_error(outcome.iloc[holdback], label)

    mean_mse = np.mean(Acc)
    print("Mean MSE for Outcome 1:", mean_mse)


    # Feature selection
    sorted_indices = np.argsort(impOOB)
    bestfeat = sorted_indices[7:]
    t = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
    MdlBF = t.fit(features.iloc[:, bestfeat], outcome)
    oob_error_bf = 1 - MdlBF.score(features.iloc[:, bestfeat], outcome)
    print("Out-of-Bag MSE (Feature Selection):", oob_error_bf)

    # Print the selected features
    selected_features = features.columns[bestfeat]
    print("Selected Features:", selected_features)




## RF for components

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn.impute import SimpleImputer


df = all_df

# Select features
features = df[['nr_of_turns', 'turns_therapist',
       'turns_speaker_patient', 'duration_therapist', 'duration_patient',
       'participation_equality', 'turn_level_freedom',
       'avgturnlength_therapist', 'avgturnlength_patient',
       'wordcount_speaker_therapist', 'wordcount_speaker_patient',
       'speechrate_speaker_therapist', 'speechrate_speaker_patient',
       'speechduration_therapist', 'speechduration_patient',
       'durationpercentage_therapist', 'durationpercentage_patient']]

# Prepare outcome variables
outcome_bond = bond_df
outcome_goal = goal_df
outcome_task = task_df

# Handle missing values in features using an imputer
imputer = SimpleImputer(strategy='mean')
features_imputed = imputer.fit_transform(features)

# Train the model for bond_df
t_bond = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
t_bond.fit(features_imputed, outcome_bond)

# Train the model for goal_df
t_goal = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
t_goal.fit(features_imputed, outcome_goal)

# Train the model for task_df
t_task = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
t_task.fit(features_imputed, outcome_task)

# Calculate out-of-bag MSE for bond_df
oob_error_bond = 1 - t_bond.score(features_imputed, outcome_bond)
print("Out-of-Bag MSE for bond_df:", oob_error_bond)

# Calculate out-of-bag MSE for goal_df
oob_error_goal = 1 - t_goal.score(features_imputed, outcome_goal)
print("Out-of-Bag MSE for goal_df:", oob_error_goal)

# Calculate out-of-bag MSE for task_df
oob_error_task = 1 - t_task.score(features_imputed, outcome_task)
print("Out-of-Bag MSE for task_df:", oob_error_task)

# Feature importances for bond_df
print("Feature importances for bond_df:", t_bond.feature_importances_)

# Feature importances for goal_df
print("Feature importances for goal_df:", t_goal.feature_importances_)

# Feature importances for task_df
print("Feature importances for task_df:", t_task.feature_importances_)

# Plots (if desired)
# Plot feature importances for bond_df
impOOB_bond = t_bond.feature_importances_
plt.bar(range(len(impOOB_bond)), impOOB_bond)
plt.title('Feature Importance for bond_df')
plt.xlabel('Predictor variable')
plt.ylabel('Importance')
plt.xticks(range(len(impOOB_bond)), features.columns, rotation=90)
plt.show()

# Plot feature importances for goal_df
impOOB_goal = t_goal.feature_importances_
plt.bar(range(len(impOOB_goal)), impOOB_goal)
plt.title('Feature Importance for goal_df')
plt.xlabel('Predictor variable')
plt.ylabel('Importance')
plt.xticks(range(len(impOOB_goal)), features.columns, rotation=90)
plt.show()

# Plot feature importances for task_df
impOOB_task = t_task.feature_importances_
plt.bar(range(len(impOOB_task)), impOOB_task)
plt.title('Feature Importance for task_df')
plt.xlabel('Predictor variable')
plt.ylabel('Importance')
plt.xticks(range(len(impOOB_task)), features.columns, rotation=90)
plt.show()

# Cross-validation for bond_df
predicted_bond = cross_val_predict(t_bond, features_imputed, outcome_bond, cv=10)
mse_bond = mean_squared_error(outcome_bond, predicted_bond)
print("Cross-validation MSE for bond_df:", mse_bond)

# Cross-validation for goal_df
predicted_goal = cross_val_predict(t_goal, features_imputed, outcome_goal, cv=10)
mse_goal = mean_squared_error(outcome_goal, predicted_goal)
print("Cross-validation MSE for goal_df:", mse_goal)

# Cross-validation for task_df
predicted_task = cross_val_predict(t_task, features_imputed, outcome_task, cv=10)
mse_task = mean_squared_error(outcome_task, predicted_task)
print("Cross-validation MSE for task_df:", mse_task)

# Mean MSE for bond_df
mean_mse_bond = np.mean(mse_bond)
print("Mean MSE for bond_df:", mean_mse_bond)

# Mean MSE for goal_df
mean_mse_goal = np.mean(mse_goal)
print("Mean MSE for goal_df:", mean_mse_goal)

# Mean MSE for task_df
mean_mse_task = np.mean(mse_task)
print("Mean MSE for task_df:", mean_mse_task)


## Pearson correlation

In [ ]:
import numpy as np
from scipy.stats import pearsonr, linregress
from tabulate import tabulate

correlation_results = []

# Calculate correlations and slopes
for feature in features:
    correlated_wai_questions = []

    for wai_question in range(1, 13):
        question_col = str(wai_question)
        val = feature

        # Calculate Pearson correlation coefficient and p-value
        correlation_coefficient, p_value = pearsonr(df[val].astype(float), df[question_col].astype(float))

        if p_value < 0.05:
            # Calculate the slope of linear regression
            slope, _, _, _, _ = linregress(df[val].astype(float), df[question_col].astype(float))
            correlated_wai_questions.append([f'WAI Question {wai_question}', f"{correlation_coefficient:.2f} ({slope:.2f})", f"{p_value:.4f}"])

    if correlated_wai_questions:
        correlation_results.append([feature, correlated_wai_questions])

# Print results in a table
if correlation_results:
    headers = ["Feature", "WAI (Factors)","Correlation Coefficient (Slope)", "p-value"]
    table_data = []
    for result in correlation_results:
        feature_name = result[0]
        for corr_data in result[1]:
            table_data.append([feature_name] + corr_data)

    table = tabulate(table_data, headers=headers, tablefmt="pretty")
    # Save the table as a vector PDF
    plt.figure(figsize=(10, 6))
    plt.axis('off')
    plt.table(cellText=table_data, colLabels=headers, cellLoc='center', loc='center', colColours=['#f5f5f5']*len(headers))
    plt.savefig('correlation_table_p.pdf', format='pdf', bbox_inches='tight')
    plt.close()

    print("Correlation Results for Patient scores Trento features:")
    print(table)
else:
    print("No correlations with p-value < 0.05 found.")


## ANOVA

In [ ]:
from scipy.stats import f_oneway
from tabulate import tabulate

anova_results = []

# Perform ANOVA test for each question and feature
for i in range(12):
    nr = str(i + 1)
    wai_scores = df[nr].astype('category')
    for feature in features:
        groups = []
        for category in wai_scores.cat.categories:
            groups.append(df[feature][wai_scores == category])

        # Perform ANOVA test
        f_statistic, p_value = f_oneway(*groups)

        if p_value < 0.05:
            anova_results.append([f'WAI Question {nr}', feature, f"{f_statistic:.2f}", f"{p_value:.4f}"])

# Print results in a table
if anova_results:
    headers = ["WAI Question", "Feature", "F-Statistic", "p-value"]
    table = tabulate(anova_results, headers=headers, tablefmt="pretty")

    # Save the table as a vector PDF
    plt.figure(figsize=(10, 6))
    plt.axis('off')
    plt.table(cellText=anova_results, colLabels=headers, cellLoc='center', loc='center', colColours=['#f5f5f5']*len(headers))
    plt.savefig('anova_table_p.pdf', format='pdf', bbox_inches='tight')
    plt.close()

    print("ANOVA Results:")
    print(table)
else:
    print("No significant ANOVA results (p-value < 0.05) found.")


## heatmap for correlation between features

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'df' is your DataFrame with the features
features = ['nr_of_turns', 'turns_therapist', 'turns_speaker_patient', 'duration_therapist', 'duration_patient',
            'participation_equality', 'turn_level_freedom', 'avgturnlength_therapist', 'avgturnlength_patient',
            'wordcount_speaker_therapist', 'wordcount_speaker_patient', 'speechrate_speaker_therapist',
            'speechrate_speaker_patient', 'speechduration_therapist', 'speechduration_patient',
            'durationpercentage_therapist', 'durationpercentage_patient']

# Create a correlation matrix
correlation_matrix = df[features].corr()

# Create a heatmap to visualize the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("heatmap correlations between features for patient WAI ratings")
plt.show()



# Therapist

In [ ]:
Therapist
#bond: 2,5,7,9
#goal: 3,4,8
#task: 1,2,10

Patient + Observer
#bond: 3,5,7,9
#goal: 1,4,8,11
#task: 2,6,10,12

In [75]:
import pandas as pd

df = pd.read_csv('/Users/rivka/Desktop/Final/Trentofeat_incl_tWAI2.csv', delimiter=',')

# List of column names
columns_to_check = ['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10']

# Convert the specified columns to numeric, replacing invalid values with NaN
df[columns_to_check] = df[columns_to_check].apply(pd.to_numeric, errors='coerce')

# Check for non-float values in the specified columns
non_float_mask = ~df[columns_to_check].applymap(lambda x: isinstance(x, float))

# Find rows where any of the specified columns have non-float values
rows_with_non_floats = non_float_mask.any(axis=1)

# Remove rows with non-float values
df = df.drop_duplicates()
df = df.dropna(subset=columns_to_check, how='any')

df = df.drop_duplicates(subset=['ppnr', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10'])

# If needed, reset the index of the filtered DataFrame
df.reset_index(drop=True, inplace=True)
df.fillna(0, inplace=True)

In [76]:
import pandas as pd


# Assuming you have a DataFrame called 'df' with the desired columns
bond_columns = ['t2', 't5', 't7','t9']
goal_columns = ['t3', 't4', 't8']
task_columns = ['t1', 't2', 't6', 't10']

# Calculate the row-wise average for the specified columns
bond_df = df[bond_columns].mean(axis=1)
goal_df = df[goal_columns].mean(axis=1)
task_df = df[task_columns].mean(axis=1)



In [ ]:
df #40 rows

## RF

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn.impute import SimpleImputer

# Load data from CSV
all_df = df

# Convert numeric columns to float
numeric_columns = ['ppnr', 'session']
all_df[numeric_columns] = all_df[numeric_columns].astype(float)

df = all_df

# Select features and target variable
features = df[[ 'mean_waittime', 'median_waittime', 'std_waittime','nr_of_turns', 'turns_therapist',
       'turns_speaker_patient', 'duration_therapist', 'duration_patient',
       'participation_equality', 'turn_level_freedom',
       'avgturnlength_therapist', 'avgturnlength_patient',
       'wordcount_speaker_therapist', 'wordcount_speaker_patient',
       'speechrate_speaker_therapist', 'speechrate_speaker_patient',
       'speechduration_therapist', 'speechduration_patient',
       'durationpercentage_therapist', 'durationpercentage_patient']]

# Prepare outcome variable
# outcome = df['WAI_score']

for i in range(10):
    nr = 't' + str(i+1) #'t' +

    outcome = all_df[nr]
    print("WAI question:", nr)

    # Handle missing values in features using an imputer
    imputer = SimpleImputer(strategy='mean')
    features_imputed = imputer.fit_transform(features)

    # Train the model
    t = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
    t.fit(features_imputed, outcome)

    # Calculate out-of-bag MSE for Outcome 1
    oob_error = 1 - t.score(features_imputed, outcome)
    print("Out-of-Bag MSE for Outcome 1:", oob_error)

    # Get feature importances
    impOOB = t.feature_importances_

    # Plot feature importances
    plt.bar(range(len(impOOB)), impOOB)
    plt.title('Unbiased Predictor Importance Estimates')
    plt.xlabel('Predictor variable')
    plt.ylabel('Importance')
    plt.xticks(range(len(impOOB)), features.columns, rotation=90)
    # plt.show()

    # Check by predicting held back data
    ncells = features.shape[0]
    perc = int(0.8 * ncells)

    nboots = 10
    Acc = np.empty(nboots)
    for b in range(nboots):
        shuf = np.random.permutation(ncells)
        incl = shuf[:perc]
        holdback = shuf[perc:]

        Mdl = t.fit(features.iloc[incl], outcome.iloc[incl])
        label = Mdl.predict(features.iloc[holdback])
        Acc[b] = mean_squared_error(outcome.iloc[holdback], label)

    mean_mse = np.mean(Acc)
    print("Mean MSE for Outcome 1:", mean_mse)

    # Feature selection
    sorted_indices = np.argsort(impOOB)
    bestfeat = sorted_indices[7:]
    t = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
    MdlBF = t.fit(features.iloc[:, bestfeat], outcome)
    oob_error_bf = 1 - MdlBF.score(features.iloc[:, bestfeat], outcome)
    print("Out-of-Bag MSE (Feature Selection):", oob_error_bf)

    # Print the selected features
    selected_features = features.columns[bestfeat]
    print("Selected Features:", selected_features)




## RF for components

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn.impute import SimpleImputer

df = all_df

# Select features
features = df[['nr_of_turns', 'turns_therapist',
       'turns_speaker_patient', 'duration_therapist', 'duration_patient',
       'participation_equality', 'turn_level_freedom',
       'avgturnlength_therapist', 'avgturnlength_patient',
       'wordcount_speaker_therapist', 'wordcount_speaker_patient',
       'speechrate_speaker_therapist', 'speechrate_speaker_patient',
       'speechduration_therapist', 'speechduration_patient',
       'durationpercentage_therapist', 'durationpercentage_patient']]

# Prepare outcome variables
outcome_bond = bond_df
outcome_goal = goal_df
outcome_task = task_df

# Handle missing values in features using an imputer
imputer = SimpleImputer(strategy='mean')
features_imputed = imputer.fit_transform(features)

# Train the model for bond_df
t_bond = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
t_bond.fit(features_imputed, outcome_bond)

# Train the model for goal_df
t_goal = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
t_goal.fit(features_imputed, outcome_goal)

# Train the model for task_df
t_task = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
t_task.fit(features_imputed, outcome_task)

# Calculate out-of-bag MSE for bond_df
oob_error_bond = 1 - t_bond.score(features_imputed, outcome_bond)
print("Out-of-Bag MSE for bond_df:", oob_error_bond)

# Calculate out-of-bag MSE for goal_df
oob_error_goal = 1 - t_goal.score(features_imputed, outcome_goal)
print("Out-of-Bag MSE for goal_df:", oob_error_goal)

# Calculate out-of-bag MSE for task_df
oob_error_task = 1 - t_task.score(features_imputed, outcome_task)
print("Out-of-Bag MSE for task_df:", oob_error_task)

# Feature importances for bond_df
print("Feature importances for bond_df:", t_bond.feature_importances_)

# Feature importances for goal_df
print("Feature importances for goal_df:", t_goal.feature_importances_)

# Feature importances for task_df
print("Feature importances for task_df:", t_task.feature_importances_)

# Plots (if desired)
# Plot feature importances for bond_df
impOOB_bond = t_bond.feature_importances_
plt.bar(range(len(impOOB_bond)), impOOB_bond)
plt.title('Feature Importance for bond_df')
plt.xlabel('Predictor variable')
plt.ylabel('Importance')
plt.xticks(range(len(impOOB_bond)), features.columns, rotation=90)
plt.show()

# Plot feature importances for goal_df
impOOB_goal = t_goal.feature_importances_
plt.bar(range(len(impOOB_goal)), impOOB_goal)
plt.title('Feature Importance for goal_df')
plt.xlabel('Predictor variable')
plt.ylabel('Importance')
plt.xticks(range(len(impOOB_goal)), features.columns, rotation=90)
plt.show()

# Plot feature importances for task_df
impOOB_task = t_task.feature_importances_
plt.bar(range(len(impOOB_task)), impOOB_task)
plt.title('Feature Importance for task_df')
plt.xlabel('Predictor variable')
plt.ylabel('Importance')
plt.xticks(range(len(impOOB_task)), features.columns, rotation=90)
plt.show()

# Cross-validation for bond_df
predicted_bond = cross_val_predict(t_bond, features_imputed, outcome_bond, cv=10)
mse_bond = mean_squared_error(outcome_bond, predicted_bond)
print("Cross-validation MSE for bond_df:", mse_bond)

# Cross-validation for goal_df
predicted_goal = cross_val_predict(t_goal, features_imputed, outcome_goal, cv=10)
mse_goal = mean_squared_error(outcome_goal, predicted_goal)
print("Cross-validation MSE for goal_df:", mse_goal)

# Cross-validation for task_df
predicted_task = cross_val_predict(t_task, features_imputed, outcome_task, cv=10)
mse_task = mean_squared_error(outcome_task, predicted_task)
print("Cross-validation MSE for task_df:", mse_task)

# Mean MSE for bond_df
mean_mse_bond = np.mean(mse_bond)
print("Mean MSE for bond_df:", mean_mse_bond)

# Mean MSE for goal_df
mean_mse_goal = np.mean(mse_goal)
print("Mean MSE for goal_df:", mean_mse_goal)

# Mean MSE for task_df
mean_mse_task = np.mean(mse_task)
print("Mean MSE for task_df:", mean_mse_task)


## Pearson Correlation

In [ ]:
import numpy as np
from scipy.stats import pearsonr, linregress
from tabulate import tabulate

correlation_results = []

# Calculate correlations and slopes
for feature in features:
    correlated_wai_questions = []

    for wai_question in range(1, 11):
        question_col = 't' + str(wai_question)
        val = feature

        # Calculate Pearson correlation coefficient and p-value
        correlation_coefficient, p_value = pearsonr(df[val].astype(float), df[question_col].astype(float))

        if p_value < 0.05:
            # Calculate the slope of linear regression
            slope, _, _, _, _ = linregress(df[val].astype(float), df[question_col].astype(float))
            correlated_wai_questions.append([f'WAI Question {wai_question}', f"{correlation_coefficient:.2f} ({slope:.2f})", f"{p_value:.4f}"])

    if correlated_wai_questions:
        correlation_results.append([feature, correlated_wai_questions])

# Print results in a table
if correlation_results:
    headers = ["Feature", "WAI (Factors)","Correlation Coefficient (Slope)", "p-value"]
    table_data = []
    for result in correlation_results:
        feature_name = result[0]
        for corr_data in result[1]:
            table_data.append([feature_name] + corr_data)

    table = tabulate(table_data, headers=headers, tablefmt="pretty")
    # Save the table as a vector PDF
    plt.figure(figsize=(10, 6))
    plt.axis('off')
    plt.table(cellText=table_data, colLabels=headers, cellLoc='center', loc='center', colColours=['#f5f5f5']*len(headers))
    plt.savefig('correlation_table_t.pdf', format='pdf', bbox_inches='tight')
    plt.close()

    print("Correlation Results for Therapist scores Trento features:")
    print(table)
else:
    print("No correlations with p-value < 0.05 found.")

## ANOVA

In [ ]:
from scipy.stats import f_oneway

for i in range(10):
  nr =  't' + str(i + 1) # 't' +
  wai_scores = df[nr].astype('category')
  for feature in features:
      groups = []
      for category in wai_scores.cat.categories:
          groups.append(df[feature][wai_scores == category])

      # Perform ANOVA test
      f_statistic, p_value = f_oneway(*groups)

      if p_value < 0.05:
          anova_results.append([f'WAI Question {nr}', feature, f"{f_statistic:.2f}", f"{p_value:.4f}"])

# Print results in a table
if anova_results:
    headers = ["WAI Question", "Feature", "F-Statistic", "p-value"]
    table = tabulate(anova_results, headers=headers, tablefmt="pretty")

    # Save the table as a vector PDF
    plt.figure(figsize=(10, 6))
    plt.axis('off')
    plt.table(cellText=anova_results, colLabels=headers, cellLoc='center', loc='center', colColours=['#f5f5f5']*len(headers))
    plt.savefig('anova_table_p.pdf', format='pdf', bbox_inches='tight')
    plt.close()

    print("ANOVA Results:")
    print(table)
else:
    print("No significant ANOVA results (p-value < 0.05) found.")



## heatmap for correlation between features

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'df' is your DataFrame with the features
features = ['nr_of_turns', 'turns_therapist', 'turns_speaker_patient', 'duration_therapist', 'duration_patient',
            'participation_equality', 'turn_level_freedom', 'avgturnlength_therapist', 'avgturnlength_patient',
            'wordcount_speaker_therapist', 'wordcount_speaker_patient', 'speechrate_speaker_therapist',
            'speechrate_speaker_patient', 'speechduration_therapist', 'speechduration_patient',
            'durationpercentage_therapist', 'durationpercentage_patient']

# Create a correlation matrix
correlation_matrix = df[features].corr()

# Create a heatmap to visualize the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Correlation Matrix Heatmap")
plt.show()


# Observer scores

In [93]:
import pandas as pd

df = pd.read_csv('allfeat_incl_O_WAI2.csv', delimiter=',')

new_column_names = {str(i): f'o{i}' for i in range(1, 13)}
df.rename(columns=new_column_names, inplace=True)

# List of column names
columns_to_check = ['o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'o7', 'o8', 'o9', 'o10','o11', 'o12']

# Convert the specified columns to numeric, replacing invalid values with NaN
df[columns_to_check] = df[columns_to_check].apply(pd.to_numeric, errors='coerce')

# Check for non-float values in the specified columns
non_float_mask = ~df[columns_to_check].applymap(lambda x: isinstance(x, float))

# Find rows where any of the specified columns have non-float values
rows_with_non_floats = non_float_mask.any(axis=1)

df = df.drop_duplicates(subset=['ppnr', 'o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'o7', 'o8', 'o9', 'o10','o11', 'o12'])
df = df.dropna(subset=columns_to_check, how='any')

# If needed, reset the index of the filtered DataFrame
df.reset_index(drop=True, inplace=True)
df.fillna(0, inplace=True)

In [89]:
import pandas as pd


# Assuming you have a DataFrame called 'df' with the desired columns
bond_columns = ['o3', 'o5', 'o7','o9']
goal_columns = ['o1', 'o4', 'o8','o11']
task_columns = ['o2', 'o6', 'o10','o12']

# Calculate the row-wise average for the specified columns
bond_df = df[bond_columns].mean(axis=1)
goal_df = df[goal_columns].mean(axis=1)
task_df = df[task_columns].mean(axis=1)

# Calculate the row-wise average for the specified columns
bond_df = df[bond_columns].mean(axis=1)
goal_df = df[goal_columns].mean(axis=1)
task_df = df[task_columns].mean(axis=1)



In [ ]:
df #68 rows

## RF

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn.impute import SimpleImputer

# Load data from CSV
all_df = df

# Convert numeric columns to float
numeric_columns = ['ppnr', 'session']
all_df[numeric_columns] = all_df[numeric_columns].astype(float)

df = all_df

# Select features and target variable
features = df[[ 'mean_waittime', 'median_waittime', 'std_waittime','nr_of_turns', 'turns_therapist',
       'turns_speaker_patient', 'duration_therapist', 'duration_patient',
       'participation_equality', 'turn_level_freedom',
       'avgturnlength_therapist', 'avgturnlength_patient',
       'wordcount_speaker_therapist', 'wordcount_speaker_patient',
       'speechrate_speaker_therapist', 'speechrate_speaker_patient',
       'speechduration_therapist', 'speechduration_patient',
       'durationpercentage_therapist', 'durationpercentage_patient']]


for i in range(12):
    nr = 'o' + str(i+1)

    outcome = all_df[nr]
    print("WAI question:", nr)

    # Handle missing values in features using an imputer
    imputer = SimpleImputer(strategy='mean')
    features_imputed = imputer.fit_transform(features)

    # Train the model
    t = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
    t.fit(features_imputed, outcome)

    # Calculate out-of-bag MSE for Outcome 1
    oob_error = 1 - t.score(features_imputed, outcome)
    print("Out-of-Bag MSE for Outcome 1:", oob_error)

    # Get feature importances
    impOOB = t.feature_importances_

    # Plot feature importances
    plt.bar(range(len(impOOB)), impOOB)
    plt.title('Unbiased Predictor Importance Estimates')
    plt.xlabel('Predictor variable')
    plt.ylabel('Importance')
    plt.xticks(range(len(impOOB)), features.columns, rotation=90)
    # plt.show()

    # Check by predicting held back data
    ncells = features.shape[0]
    perc = int(0.8 * ncells)

    nboots = 10
    Acc = np.empty(nboots)
    for b in range(nboots):
        shuf = np.random.permutation(ncells)
        incl = shuf[:perc]
        holdback = shuf[perc:]

        Mdl = t.fit(features.iloc[incl], outcome.iloc[incl])
        label = Mdl.predict(features.iloc[holdback])
        Acc[b] = mean_squared_error(outcome.iloc[holdback], label)

    mean_mse = np.mean(Acc)
    print("Mean MSE for Outcome 1:", mean_mse)

    # Feature selection
    sorted_indices = np.argsort(impOOB)
    bestfeat = sorted_indices[7:]
    t = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
    MdlBF = t.fit(features.iloc[:, bestfeat], outcome)
    oob_error_bf = 1 - MdlBF.score(features.iloc[:, bestfeat], outcome)
    print("Out-of-Bag MSE (Feature Selection):", oob_error_bf)

    # Print the selected features
    selected_features = features.columns[bestfeat]
    print("Selected Features:", selected_features)




## RF for components

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn.impute import SimpleImputer

df = all_df

# Select features
features = df[['nr_of_turns', 'turns_therapist',
       'turns_speaker_patient', 'duration_therapist', 'duration_patient',
       'participation_equality', 'turn_level_freedom',
       'avgturnlength_therapist', 'avgturnlength_patient',
       'wordcount_speaker_therapist', 'wordcount_speaker_patient',
       'speechrate_speaker_therapist', 'speechrate_speaker_patient',
       'speechduration_therapist', 'speechduration_patient',
       'durationpercentage_therapist', 'durationpercentage_patient']]
# Prepare outcome variables
outcome_bond = bond_df
outcome_goal = goal_df
outcome_task = task_df

# Handle missing values in features using an imputer
imputer = SimpleImputer(strategy='mean')
features_imputed = imputer.fit_transform(features)

# Train the model for bond_df
t_bond = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
t_bond.fit(features_imputed, outcome_bond)

# Train the model for goal_df
t_goal = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
t_goal.fit(features_imputed, outcome_goal)

# Train the model for task_df
t_task = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1)
t_task.fit(features_imputed, outcome_task)

# Calculate out-of-bag MSE for bond_df
oob_error_bond = 1 - t_bond.score(features_imputed, outcome_bond)
print("Out-of-Bag MSE for bond_df:", oob_error_bond)

# Calculate out-of-bag MSE for goal_df
oob_error_goal = 1 - t_goal.score(features_imputed, outcome_goal)
print("Out-of-Bag MSE for goal_df:", oob_error_goal)

# Calculate out-of-bag MSE for task_df
oob_error_task = 1 - t_task.score(features_imputed, outcome_task)
print("Out-of-Bag MSE for task_df:", oob_error_task)

# Feature importances for bond_df
print("Feature importances for bond_df:", t_bond.feature_importances_)

# Feature importances for goal_df
print("Feature importances for goal_df:", t_goal.feature_importances_)

# Feature importances for task_df
print("Feature importances for task_df:", t_task.feature_importances_)

# Plots (if desired)
# Plot feature importances for bond_df
impOOB_bond = t_bond.feature_importances_
plt.bar(range(len(impOOB_bond)), impOOB_bond)
plt.title('Feature Importance for bond_df')
plt.xlabel('Predictor variable')
plt.ylabel('Importance')
plt.xticks(range(len(impOOB_bond)), features.columns, rotation=90)
plt.show()

# Plot feature importances for goal_df
impOOB_goal = t_goal.feature_importances_
plt.bar(range(len(impOOB_goal)), impOOB_goal)
plt.title('Feature Importance for goal_df')
plt.xlabel('Predictor variable')
plt.ylabel('Importance')
plt.xticks(range(len(impOOB_goal)), features.columns, rotation=90)
plt.show()

# Plot feature importances for task_df
impOOB_task = t_task.feature_importances_
plt.bar(range(len(impOOB_task)), impOOB_task)
plt.title('Feature Importance for task_df')
plt.xlabel('Predictor variable')
plt.ylabel('Importance')
plt.xticks(range(len(impOOB_task)), features.columns, rotation=90)
plt.show()

# Cross-validation for bond_df
predicted_bond = cross_val_predict(t_bond, features_imputed, outcome_bond, cv=10)
mse_bond = mean_squared_error(outcome_bond, predicted_bond)
print("Cross-validation MSE for bond_df:", mse_bond)

# Cross-validation for goal_df
predicted_goal = cross_val_predict(t_goal, features_imputed, outcome_goal, cv=10)
mse_goal = mean_squared_error(outcome_goal, predicted_goal)
print("Cross-validation MSE for goal_df:", mse_goal)

# Cross-validation for task_df
predicted_task = cross_val_predict(t_task, features_imputed, outcome_task, cv=10)
mse_task = mean_squared_error(outcome_task, predicted_task)
print("Cross-validation MSE for task_df:", mse_task)

# Mean MSE for bond_df
mean_mse_bond = np.mean(mse_bond)
print("Mean MSE for bond_df:", mean_mse_bond)

# Mean MSE for goal_df
mean_mse_goal = np.mean(mse_goal)
print("Mean MSE for goal_df:", mean_mse_goal)

# Mean MSE for task_df
mean_mse_task = np.mean(mse_task)
print("Mean MSE for task_df:", mean_mse_task)


## Pearson Correlation

In [ ]:
import numpy as np
from scipy.stats import pearsonr, linregress
from tabulate import tabulate

correlation_results = []

# Calculate correlations and slopes
for feature in features:
    correlated_wai_questions = []

    for wai_question in range(1, 13):
        question_col = 'o' + str(wai_question)
        val = feature

        # Calculate Pearson correlation coefficient and p-value
        correlation_coefficient, p_value = pearsonr(df[val].astype(float), df[question_col].astype(float))

        if p_value < 0.05:
            # Calculate the slope of linear regression
            slope, _, _, _, _ = linregress(df[val].astype(float), df[question_col].astype(float))
            correlated_wai_questions.append([f'WAI Question {wai_question}', f"{correlation_coefficient:.2f} ({slope:.2f})", f"{p_value:.4f}"])

    if correlated_wai_questions:
        correlation_results.append([feature, correlated_wai_questions])

# Print results in a table
if correlation_results:
    headers = ["Feature", "WAI (Factors)","Correlation Coefficient (Slope)", "p-value"]
    table_data = []
    for result in correlation_results:
        feature_name = result[0]
        for corr_data in result[1]:
            table_data.append([feature_name] + corr_data)

    table = tabulate(table_data, headers=headers, tablefmt="pretty")
    # Save the table as a vector PDF
    plt.figure(figsize=(10, 6))
    plt.axis('off')
    plt.table(cellText=table_data, colLabels=headers, cellLoc='center', loc='center', colColours=['#f5f5f5']*len(headers))
    plt.savefig('correlation_table_t.pdf', format='pdf', bbox_inches='tight')
    plt.close()

    print("Correlation Results for the observer scores Trento features:")
    print(table)
else:
    print("No correlations with p-value < 0.05 found.")

## ANOVA

In [ ]:
from scipy.stats import f_oneway
from tabulate import tabulate

anova_results = []

# Perform ANOVA test for each question and feature
for i in range(12):
    nr = 'o'+ str(i + 1)  # 't' +
    wai_scores = df[nr].astype('category')
    for feature in features:
        groups = []
        for category in wai_scores.cat.categories:
            groups.append(df[feature][wai_scores == category])

        # Perform ANOVA test
        f_statistic, p_value = f_oneway(*groups)

        if p_value < 0.05:
            anova_results.append([f'WAI Question {nr}', feature, f"{f_statistic:.2f}", f"{p_value:.4f}"])

# Print results in a table
if anova_results:
    headers = ["WAI Question", "Feature", "F-Statistic", "p-value"]
    table = tabulate(anova_results, headers=headers, tablefmt="pretty")

    # Save the table as a vector PDF
    plt.figure(figsize=(10, 6))
    plt.axis('off')
    plt.table(cellText=anova_results, colLabels=headers, cellLoc='center', loc='center', colColours=['#f5f5f5']*len(headers))
    plt.savefig('anova_table_p.pdf', format='pdf', bbox_inches='tight')
    plt.close()

    print("ANOVA Results:")
    print(table)
else:
    print("No significant ANOVA results (p-value < 0.05) found.")


## heatmap for correlation between features

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'df' is your DataFrame with the features
features = ['nr_of_turns', 'turns_therapist', 'turns_speaker_patient', 'duration_therapist', 'duration_patient',
            'participation_equality', 'turn_level_freedom', 'avgturnlength_therapist', 'avgturnlength_patient',
            'wordcount_speaker_therapist', 'wordcount_speaker_patient', 'speechrate_speaker_therapist',
            'speechrate_speaker_patient', 'speechduration_therapist', 'speechduration_patient',
            'durationpercentage_therapist', 'durationpercentage_patient']

# Create a correlation matrix
correlation_matrix = df[features].corr()

# Create a heatmap to visualize the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Correlation Matrix Heatmap")
plt.show()
